<a href="https://colab.research.google.com/github/JulioLino/ProjetoMestradoSaudePublica/blob/main/MestradoEstatisticaEtapaA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Analise estatistica de regreção dos dados Etapa A do mestrado da Fabiana

Importando bibliotecas

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
from google.colab import files
import io
import numpy as np
import statsmodels.api as sm
from statsmodels.miscmodels.ordinal_model import OrderedModel
import re # Para limpeza de strings

Lendo arquivo Etapa A.xlsx

In [ ]:
# --- 1. Carregar o arquivo Excel para o Google Colab ---
print("Por favor, selecione o arquivo 'Etapa A.xlsx' para upload:")
# Ao executar esta célula, uma caixa de diálogo aparecerá para você selecionar o arquivo.
uploaded = files.upload()

Por favor, selecione o arquivo 'Etapa A.xlsx' para upload:


Saving Etapa A.xlsx to Etapa A.xlsx


Analisando o arquivo

In [ ]:
file_name = 'Etapa A.xlsx' # O nome do arquivo .xlsx
if file_name not in uploaded:
    print(f"Erro: O arquivo '{file_name}' não foi encontrado entre os uploads. Verifique o nome.")
    print("Se você carregou um arquivo com nome diferente, por favor, ajuste 'file_name' no código.")
    df = None # Define df como None para evitar erros posteriores
else:
    try:
        # Lê o arquivo Excel da aba 'Planilha2'
        df = pd.read_excel(io.BytesIO(uploaded[file_name]), sheet_name='Planilha2')
        print(f"\nArquivo '{file_name}' da aba 'Planilha2' carregado com sucesso.")
    except Exception as e:
        print(f"\nErro ao carregar o arquivo Excel: {e}")
        print("Certifique-se de que o nome do arquivo ('Etapa A.xlsx') e o nome da aba ('Planilha2') estão corretos.")
        df = None

if df is not None:
    # --- 2. Renomear as colunas conforme a nova estrutura ---
    # IMPORTANTE: Verifique se as chaves deste dicionário correspondem EXATAMENTE aos cabeçalhos das colunas
    # no seu Excel na 'Planilha2'. Inclua espaços, acentos, pontuação, etc., se existirem.
    columns_mapping = {
        'Id': 'id',
        'Nivel Segurança': 'seguranca', # Nome da coluna da variável dependente
        'Leitura de artigos, anais, convenções ou livros': 'conhecimento_leitura_artigos',
        'Leitura de documentos e publicações do Ministério da Saúde e ANVISA': 'conhecimento_leitura_docs_ms_anvisa',
        'Nunca obtive conhecimentos específicos relacionados à transfusão de hemocomponentes': 'conhecimento_nunca_obtive',
        'Outros': 'conhecimento_outros',
        'Perguntando a outro profissional': 'conhecimento_perguntando_profissional',
        'Pesquisa em site de busca na internet (google, bing, inteligências artificiais)': 'conhecimento_pesquisa_internet',
        'Sendo orientado pelo preceptor (staff)': 'conhecimento_orientado_preceptor',
        'Alguma Aula': 'conhecimento_alguma_aula',
        'Algum Treinamentos': 'conhecimento_algum_treinamentos'
    }


Arquivo 'Etapa A.xlsx' da aba 'Planilha2' carregado com sucesso.


Transformação

In [ ]:
# Verifica se todas as colunas especificadas existem no DataFrame antes de renomear
missing_cols = [col for col in columns_mapping.keys() if col not in df.columns]
if missing_cols:
    print(f"\nErro: As seguintes colunas esperadas NÃO foram encontradas no DataFrame: {missing_cols}")
    print(f"Colunas presentes no arquivo: {df.columns.tolist()}")
    print("Por favor, verifique se os nomes das colunas no seu Excel correspondem EXATAMENTE aos nomes no dicionário 'columns_mapping'.")
    df = None # Impede a continuação se colunas críticas estiverem faltando
else:
    df.rename(columns=columns_mapping, inplace=True)
    print("\nVisualizando as primeiras linhas do DataFrame após renomear colunas:")
    print(df.head())
    print(f"\nColunas disponíveis após renomear: {df.columns.tolist()}")

if df is not None:
    # --- 3. Preparar as variáveis independentes (conhecimento adquirido) ---
    # As colunas de conhecimento já vêm separadas com 'Sim'/'Não'.
    # Precisamos convertê-las para valores numéricos binários (1s e 0s).
    independent_vars_prefix = 'conhecimento_' # Prefixo comum para as colunas de conhecimento
    # Filtra as colunas que começam com o prefixo 'conhecimento_'
    dummy_cols = [col for col in df.columns if col.startswith(independent_vars_prefix)]

    print(f"\nIdentificadas {len(dummy_cols)} variáveis independentes (conhecimento adquirido).")

    for col in dummy_cols:
        # Converte 'Sim' para 1, 'Não' para 0. Valores NaN serão preenchidos com 0 (assumindo 'Não').
        df[col] = df[col].replace({'Sim': 1, 'Não': 0}).fillna(0).astype(int)

    # --- 4. Codificar a variável 'seguranca' para numérica ordinal ---
    # É CRUCIAL que este mapeamento esteja na ordem CORRETA do menor para o maior nível de segurança.
    # Verifique se as strings das categorias de segurança aqui correspondem EXATAMENTE às do seu arquivo Excel.
    security_mapping = {
        'Totalmente inseguro': 1,
        'Parcialmente inseguro': 2,
        'Nem seguro e nem inseguro': 3,
        'Parcialmente seguro': 4,
        'Totalmente seguro': 5
    }

    # Aplica o mapeamento. Categorias em 'seguranca' que não estão no 'security_mapping' resultarão em NaN.
    df['seguranca_codificada'] = df['seguranca'].map(security_mapping)

    # Remover linhas com valores NaN na variável dependente codificada ou nas variáveis independentes (dummies)
    # Isso é essencial para que o modelo de regressão funcione.
    df_model = df.dropna(subset=['seguranca_codificada'] + dummy_cols)

    if df_model.empty:
        print("\nErro: Não há dados suficientes após a codificação da segurança e remoção de NaNs para a modelagem.")
        print("Verifique se as categorias de 'Nivel Segurança' em seu arquivo correspondem exatamente às do 'security_mapping'.")
    else:
        # Definir X (variáveis independentes) e y (variável dependente)
        X = df_model[dummy_cols]
        y = df_model['seguranca_codificada']

        # Remover colunas de X que são constantes (todos 0 ou todos 1) após a filtragem
        # Isso pode causar problemas de multicolinearidade ou singularidade no modelo.
        # .loc[:, (X != X.iloc[0]).any()] seleciona apenas as colunas onde há variação nos dados.
        X = X.loc[:, (X != X.iloc[0]).any()]

        if X.empty: # Verifica se X está vazio após remover colunas constantes
            print("\nErro: O DataFrame de variáveis independentes (X) está vazio ou não tem preditores variáveis após a filtragem. Não é possível rodar o modelo.")
        elif y.nunique() < 2: # O modelo precisa de pelo menos 2 categorias únicas na variável dependente
            print(f"\nErro: A variável dependente 'seguranca_codificada' tem menos de 2 categorias únicas ({y.nunique()}). Não é possível rodar o modelo.")
        else:
            # --- 5. Executar a Regressão Logística Ordinal com statsmodels ---
            print("\n--- Treinando o Modelo de Regressão Logística Ordinal ---")
            print("Isso pode levar alguns segundos, ou falhar se os dados tiverem problemas de separabilidade ou pouca variação.")

            try:
                # O OrderedModel do statsmodels já inclui o intercepto (constante) automaticamente.
                # Por isso, passamos 'X' diretamente, sem 'sm.add_constant()'.
                model = OrderedModel(y, X, distr='logit')

                # Ajustar o modelo.
                # 'method='bfgs'' é um algoritmo de otimização robusto.
                # 'cov_type='HC1'' calcula erros padrão robustos, útil para lidar com heteroscedasticidade.
                # 'maxiter' pode ser ajustado se o modelo não convergir.
                result = model.fit(method='bfgs', maxiter=1000, cov_type='HC1')

                print("\nModelo de Regressão Logística Ordinal Treinado com Sucesso.")
                print("\n--- Resumo do Modelo de Regressão Logística Ordinal ---")
                print(result.summary())

                print("\n--- Interpretação dos Resultados Principais ---")
                print("1.  **Coluna 'P>|z|'**: Este é o p-valor para cada 'Meio de Conhecimento' e para os interceptos ('const' na tabela).")
                print("    - Se **P>|z| < 0.05** (ou seu nível de significância escolhido, ex: 0.01 ou 0.10):")
                print("      Aquele 'Meio de Conhecimento' tem uma **influência estatisticamente significativa** na 'Autodeclaração de Segurança', controlando pelos outros meios.")
                print("    - Se P>|z| >= 0.05:")
                print("      Não há evidências estatísticas suficientes para afirmar uma influência significativa daquele meio de conhecimento.")
                print("\n2.  **Coluna 'coef' (coeficiente)**:")
                print("    - Um coeficiente **positivo** para um 'Meio de Conhecimento' indica que ter esse conhecimento adquirido aumenta a chance de passar para uma categoria de segurança MAIS ALTA (por exemplo, de 'Parcialmente seguro' para 'Totalmente seguro').")
                print("    - Um coeficiente **negativo** indica o oposto: diminui a chance de passar para uma categoria de segurança mais alta.")
                print("    - Os coeficientes 'const' são os *cut-points* (pontos de corte), que separam as categorias da sua variável dependente ordinal na escala latente do modelo. Eles são importantes para o modelo, mas a interpretação principal está nos coeficientes dos preditores (meios de conhecimento).")
                print("\nPara mais detalhes e diagnóstico de problemas, consulte a documentação do `statsmodels` para `OrderedModel` online.")

            except Exception as e:
                print(f"\nErro ao ajustar o modelo de Regressão Logística Ordinal: {e}")
                print("Possíveis causas comuns de falha na convergência ou erro do modelo:")
                print("  - **Problemas de Convergência:** O algoritmo pode não ter encontrado uma solução ótima.")
                print("    - Tente aumentar `maxiter` (por exemplo, para 5000).")
                print("    - Tente mudar o método de otimização (por exemplo, `method='cg'` ou `method='newton'`).")
                print("  - **Multicolinearidade:** Se duas ou mais variáveis de 'Conhecimento adquirido' são altamente correlacionadas (os participantes que têm um, quase sempre têm o outro), o modelo pode ter dificuldade em separar seus efeitos.")
                print("    - Considere remover uma das variáveis muito correlacionadas ou combiná-las se fizer sentido teórico.")
                print("  - **Separação Completa ou Quase Completa:** Se um meio de conhecimento sempre (ou quase sempre) está associado a uma única categoria extrema de segurança (ex: todos que usaram 'Aulas' são *sempre* 'Totalmente Seguro'). Isso pode causar estimativas infinitas ou erros.")
                print("  - **Categorias com pouquíssimas observações:** Se algumas combinações de 'Conhecimento adquirido' (Sim/Não) e 'Nivel Segurança' têm contagens muito baixas, o modelo pode ter dificuldade em estimar os coeficientes de forma estável.")
                print("    - Verifique a distribuição das suas variáveis. Se houver categorias com N muito baixo, agrupe-as ou considere remover a variável se for o caso.")
                print("  - **Verifique seus dados de entrada:** Certifique-se de que não há valores inesperados (além de 'Sim'/'Não' ou das categorias de segurança) nas colunas usadas.")


Visualizando as primeiras linhas do DataFrame após renomear colunas:
   id            seguranca conhecimento_leitura_artigos  \
0   7  Parcialmente seguro                          Sim   
1   8  Parcialmente seguro                          Sim   
2   9    Totalmente seguro                          Sim   
3  11  Parcialmente seguro                          Não   
4  14    Totalmente seguro                          Não   

  conhecimento_leitura_docs_ms_anvisa conhecimento_nunca_obtive  \
0                                 Sim                       Não   
1                                 Sim                       Não   
2                                 Sim                       Não   
3                                 Não                       Não   
4                                 Não                       Não   

  conhecimento_outros conhecimento_perguntando_profissional  \
0                 Não                                   Não   
1                 Não                         

/tmp/ipython-input-7-696703311.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace({'Sim': 1, 'Não': 0}).fillna(0).astype(int)
/tmp/ipython-input-7-696703311.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace({'Sim': 1, 'Não': 0}).fillna(0).astype(int)
/tmp/ipython-input-7-696703311.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt

Optimization terminated successfully.
         Current function value: 1.001274
         Iterations: 73
         Function evaluations: 74
         Gradient evaluations: 74

Modelo de Regressão Logística Ordinal Treinado com Sucesso.

--- Resumo do Modelo de Regressão Logística Ordinal ---
                              OrderedModel Results                              
Dep. Variable:     seguranca_codificada   Log-Likelihood:                -56.071
Model:                     OrderedModel   AIC:                             138.1
Method:              Maximum Likelihood   BIC:                             164.5
Date:                  Wed, 25 Jun 2025                                         
Time:                          18:57:48                                         
No. Observations:                    56                                         
Df Residuals:                        43                                         
Df Model:                             9                       

Salvar o resultado em um arquivo

In [ ]:
# prompt: Salvar resultado em um arquivo

# Convert the summary to a string
results_summary_string = str(result.summary())

# Define the output file name
output_filename = 'regressao_ordinal_resultado.txt'

# Save the string to a file
with open(output_filename, 'w') as f:
    f.write(results_summary_string)

print(f"\nResultados do modelo salvos em '{output_filename}'")

# Optional: Download the file
# files.download(output_filename)
# print(f"Arquivo '{output_filename}' pronto para download.")


Resultados do modelo salvos em 'regressao_ordinal_resultado.txt'
